In [6]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder
import pickle

In [7]:
data=pd.read_csv('Churn_Modelling.csv')
data.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [8]:
## Preprocess the data
data=data.drop(['RowNumber','CustomerId','Surname'],axis=1)

In [9]:
## Encoded categorical variables
label_encoder_gender=LabelEncoder()
data['Gender']=label_encoder_gender.fit_transform(data['Gender'])

In [10]:
## One-hot encode 'Geography'
onehot_encoder_geo=OneHotEncoder(handle_unknown='error')
geo_encoded=onehot_encoder_geo.fit_transform(data[['Geography']]).toarray()
geo_encoded_df=pd.DataFrame(geo_encoded, columns=onehot_encoder_geo.get_feature_names_out(['Geography']))
geo_encoded_df

,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0
1,0.0,0.0,1.0
2,1.0,0.0,0.0
3,1.0,0.0,0.0
4,0.0,0.0,1.0
...,...,...,...
9995,1.0,0.0,0.0
9996,1.0,0.0,0.0
9997,1.0,0.0,0.0
9998,0.0,1.0,0.0


In [11]:
## Combine one-hot encoded columns with original data
data=pd.concat([data.drop('Geography',axis=1),geo_encoded_df],axis=1)
data.head()

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,0,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0.0,0.0,1.0


In [12]:
## Split the data into features and target
X=data.drop('EstimatedSalary',axis=1)
y=data['EstimatedSalary']

In [13]:
## Split the data in training and testing sets
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=42)

In [14]:
## Scale these features
scaler=StandardScaler()
X_train=scaler.fit_transform(X_train)
X_test=scaler.transform(X_test)

In [15]:
## Save the encoder and scaler for later use
with open('label_encoder_gender.pkl','wb') as file:
    pickle.dump(label_encoder_gender,file)

with open('onehot_encoder_geo.pkl','wb') as file:
    pickle.dump(onehot_encoder_geo,file)

with open('scaler.pkl','wb') as file:
    pickle.dump(scaler,file)

## ANN Regression Problem Statement

In [16]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

In [17]:
## Build the model
model=Sequential([
    Dense(64,activation='relu',input_shape=(X_train.shape[1],)),
    Dense(32, activation='relu'),
    Dense(1) # Output layer for regression
])

In [18]:
## Compile the model
model.compile(optimizer='adam',loss='mean_absolute_error',metrics=['mae'])

In [19]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 64)                832       
                                                                 
 dense_1 (Dense)             (None, 32)                2080      
                                                                 
 dense_2 (Dense)             (None, 1)                 33        
                                                                 
Total params: 2945 (11.50 KB)
Trainable params: 2945 (11.50 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [20]:
## Training the model

from tensorflow.keras.callbacks import EarlyStopping, TensorBoard
import datetime

# set up tensorboard
log_dir = "regressionlogs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = TensorBoard(log_dir=log_dir,histogram_freq=1)

In [21]:
# Set up Early Stopping
early_stopping_callback=EarlyStopping(monitor='val_loss',patience=10,restore_best_weights=True)


In [22]:
# Train the model
history=model.fit(
    X_train,y_train,
    validation_data=(X_test,y_test),
    epochs=100,
    callbacks=[early_stopping_callback,tensorboard_callback]
)

Epoch 1/100


250/250 [==============================] - 4s 7ms/step - loss: 100368.3125 - mae: 100368.3125 - val_loss: 98481.2188 - val_mae: 98481.2188
Epoch 2/100
250/250 [==============================] - 1s 4ms/step - loss: 99497.4766 - mae: 99497.4766 - val_loss: 96731.5703 - val_mae: 96731.5703
Epoch 3/100
250/250 [==============================] - 1s 5ms/step - loss: 96459.0703 - mae: 96459.0703 - val_loss: 92294.8516 - val_mae: 92294.8516
Epoch 4/100
250/250 [==============================] - 1s 5ms/step - loss: 90561.1875 - mae: 90561.1875 - val_loss: 85069.9688 - val_mae: 85069.9688
Epoch 5/100
250/250 [==============================] - 1s 4ms/step - loss: 82157.8438 - mae: 82157.8438 - val_loss: 75977.5703 - val_mae: 75977.5703
Epoch 6/100
250/250 [==============================] - 1s 4ms/step - loss: 72639.3672 - mae: 72639.3672 - val_loss: 66986.2500 - val_mae: 66986.2500
Epoch 7/100
250/250 [==============================] - 1s 4ms/step - loss: 63874.1133 - mae: 63874.113

In [23]:
%load_ext tensorboard

In [24]:
%tensorboard --logdir regressionlogs/fit

In [25]:
## Evaluate model on the test data
test_loss,test_mae=model.evaluate(X_test,y_test)
print(f'Test MAE : {test_mae}')

63/63 [==============================] - 0s 2ms/step - loss: 50283.9375 - mae: 50283.9375
Test MAE : 50283.9375


In [26]:
model.save('regression_model.h5')

c:\Users\LENOVO\Desktop\Portfolio_AI\P7_Regression_ChurnModelling_ANN\venv\Lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
